<a href="https://colab.research.google.com/github/ayush12gupta/Probabilistic-ML-ACA-/blob/master/BMF_MCMC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
# Importing Train File
from google.colab import files
uploaded = files.upload()


Saving u1.base to u1.base


In [8]:
# Importing Test File
from google.colab import files
uploaded = files.upload()

Saving u1.test to u1.test


In [0]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import wishart

In [0]:
N = 943 # No. of users
M = 1682 # No. of movies  
D = 50 

In [0]:
def matrix_generator(N,M,file):

    I = np.zeros((N,M))
    R = np.zeros((N,M))
    spt = "	"
    op = open(file,"r")
    for line in op:
        (usr,movie,rating,_) = line.split(spt)
        #check !!!!!!!!
        R[int(usr) - 1, int(movie) - 1] = int(rating)
        I[int(usr) - 1, int(movie) - 1] = 1.0
    return R, I

In [0]:
R, I = matrix_generator(N,M,"u1.base")
RTest, ITest = matrix_generator(N,M,"u1.test")

**Hyperparameter Declaration**

In [0]:
w0 = np.eye(D)
mu0 = np.zeros(D)
v0 = D
alpha = 2.0
beta0 = 1.0


In [0]:
lambv = wishart.rvs(size = 1,df = vv0, scale = wv0)
lambu = wishart.rvs(size = 1,df = vu0, scale = wu0)
muu = np.random.multivariate_normal(muu0, np.linalg.pinv(beta0 * lambu))
muv = np.random.multivariate_normal(muv0, np.linalg.pinv(beta0 * lambv))
U = np.zeros((N,D))  
V = np.zeros((M,D))
U = np.random.multivariate_normal(muu,lambu,N)
V = np.random.multivariate_normal(muv,lambv,M)
sigmaUN = np.zeros((N, D, D))
sigmaVM = np.zeros((M, D, D))
muun = np.zeros((N, D))
muvm = np.zeros((M, D))
vvt = np.zeros((M , D, D))
uut = np.zeros((N , D, D))
for i in range(M):
  vvt[i] = V[i, :].reshape(D, 1) @ V[i ,:].reshape(1,D)
for i in range(N):
  uut[i] = U[i, :].reshape(D, 1) @ U[i ,:].reshape(1,D)  
 

In [0]:
num = 100
U_spl = np.zeros((num ,N , D))
V_spl = np.zeros((num ,M ,D))
for t in range(num):
  
  #Calculating joint posterior for lambdaU and muU
  
  ubar = np.sum(U ,axis = 0) / N
  #print(ubar,"bar")
  subar = np.zeros((D ,D))
  for i in range(N):
    subar += U[i ,:].reshape(D ,1) @ U[i ,:].reshape(1 ,D)
  subar /= N
  #print(subar,"subar")
  betastaru = beta0 + N
  muustar = (beta0 * mu0 + N * ubar) / betastaru  
  #print(muustar,"!!!!")
  vustar = v0 + N
  wustr0 = np.linalg.pinv(np.linalg.pinv(w0) + N * subar + beta0 * N * (mu0 - ubar).reshape(D ,1) @ (mu0 - ubar).reshape(1 ,D) / betastaru)
  
 #Calculating joint posterior for lambdaV and muV
  
  vbar = np.sum(V ,axis = 0) / M
  svbar = np.zeros((D , D))
  for j in range(M):
    svbar += V[i ,:].reshape(D ,1) @ V[i ,:].reshape(1 ,D)
  svbar /= M
  muvstar = (beta0 * mu0 + M * vbar) / (beta0 + M) #muv0 pass
  #print(muvstar,"1234")
  betastrv = beta0 + M
  vvstar = v0 + M
  wvstr0 = np.linalg.pinv(np.linalg.pinv(w0) + M * svbar + beta0 * M * (mu0 - vbar).reshape(D ,1) @ (mu0 - vbar).reshape(1 ,D) / betastrv)
 
  # Sampling lambaU and muU
  
  lambu = wishart.rvs(size = 1,df = vustar, scale = wustr0)
  muu = np.random.multivariate_normal(muustar, np.linalg.pinv(betastaru * lambu))
  #print(lambu,"lambu")
  #print(muu,"mu")
   
  # Sampling lambdaV and muV
  
  lambv = wishart.rvs(size = 1,df = vvstar, scale = wvstr0)
  muv = np.random.multivariate_normal(muvstar, np.linalg.pinv(betastrv * lambv))
  #print(muv,"muv")
  
  #Calculating Conditional Posterior for U
  
  for i in range(N):
    s1 = np.zeros((D, D))
    s2 = np.zeros(D)
    for j in range(M):
      s1 += I[i][j] * vvt[j]
      s2 += R[i][j] * V[j ,:]
    s2 *= alpha
    sigmaUN[i] = np.linalg.pinv(lambu + alpha * s1)
    muun[i] = sigmaUN[i] @ ( s2 + lambu @ muu)
    
  #Sampling U
  
  for i in range(N):
    U[i] = np.random.multivariate_normal(muun[i], sigmaUN[i])
  U_spl[t] = U
  #print(U_spl[t])
  for i in range(N):
    uut[i] = U[i, :].reshape(D, 1) @ U[i ,:].reshape(1,D)
    
  #Calculating Conditional Posterior for V
  
  for j in range(M):#
    A = np.zeros((D ,D))
    B = np.zeros(D)
    for i in range(N):
      A += I[i][j] * uut[i]
      B += R[i][j] * U[i ,:]
    B *= alpha
    sigmaVM[j] = np.linalg.pinv(lambv + alpha * A)
    muvm[j] = sigmaVM[j] @ (B + lambv @ muv)
    
  #Sampling V
  
  for j in range(M):
    V[j] = np.random.multivariate_normal(muvm[j] ,sigmaVM[j])
  V_spl[t] = V
  #print(V_spl[t])
  for i in range(M):
    vvt[i] = V[i, :].reshape(D, 1) @ V[i ,:].reshape(1, D)
    
  print (t + 1)
  predictR = np.zeros((N,M))
  predictR += U_spl[t]@V_spl[t].T
  #print(predictR)
  RMSE = (np.sum(((ITest*predictR) - RTest) ** 2) / np.sum(ITest)) ** 0.5
  print(RMSE)

1
5.920613308822863
2
1.4707984611868525
3
1.0231974995193707
4
1.0150364654538937
5
1.0156753062154362
6
1.0288489979147588
7
1.0217198500795015
8
1.0145596372864243
9
1.0278115957983862
10
1.0293119925407916
11
1.0416835858029616
12
1.033870651584277
13
1.0373943327482311
14
1.0360493648450388
15
1.0447931088026012
16
1.0506483427840043
17
1.045830370104952
18
1.0429110683886613
19
1.0418252931768759
20
1.0509166957937979
21
1.054067451159698
22
1.045856464566836
23
1.0488898986638941
24
1.0532168422408785
25
1.0409634191145367
26
1.0504744446020953
27
1.0520367953513543
28
1.053202278311832
29
1.0581523503623895
30
1.0495016644441029
31
1.052708003379424
32
1.0594354937194164
33
1.0570331339844274
34
1.0519092389614924
35
1.0516170314466144
36
1.0585206105888776
37
1.0433409934097746
38
1.0619878894441754
39
1.0533186026335335
40
1.0571418975847358
41
1.0472438764331586
42
1.0487649778831536
43
1.0526607120357576
44
1.057935897662006


In [0]:
predictR = np.zeros((N,M))
for i in range(num):
    predictR += U_spl[i]@V_spl[i].T
predictR/=num
predCheck = ITest * predictR


In [0]:
RMSE=0
RMSE = (np.sum((predCheck - RTest) ** 2) / np.sum(ITest)) ** 0.5
RMSE

0.9197090006938149

In [0]:
def sample_mu(size,uv,lamb,mu):
    bar = np.sum(uv ,axis = 0) / size
    #print(bar,"bar")
    sbar = np.zeros((D ,D))
    for i in range(size):
        sbar += uv[i ,:].reshape(D ,1) @ uv[i ,:].reshape(1 ,D) 
    sbar /= size
    #print(subar,"subar")
    mustar = (beta0 * muu0 + size * bar) / (beta0 + size)
    #print(mustar,"muuuu")
    betastar = beta0 + size
    vstar = vu0 + size
    wstr0 = np.linalg.pinv(np.linalg.pinv(wu0) + N * sbar + beta0*size*(muu0 - bar).reshape(D ,1) @ (muu0 - bar).reshape(1 ,D) / betastar)

    lamb = wishart.rvs(size = 1,df = vstar, scale = wstr0)
    #print(lamb,"lambu")
    mu = np.random.multivariate_normal(mustar, np.linalg.pinv(betastar * lamb))
    print(mu,"mu")

    return lamb,mu


In [0]:
def sample_u(lamb,mu,uv,uv2,size,size2):
    sigma = np.zeros((size, D, D))
    mun = np.zeros((size, D))
    for i in range(size): 
      s1 = np.zeros((D,D))
      s2 = np.zeros(D)
      for j in range(size2):
        s1 += I[i][j] * uv2[i, :].reshape(D, 1) @ uv2[i ,:].reshape(1,D)#vut[j]
        s2 += R[i][j] * uv2[j ,:]#uv2[i, :].reshape(D, 1) @ U[i ,:].reshape(1,D)
#uv2[j ,:]
      
      s2 *= alpha
      sigma[i] = np.linalg.pinv(lamb + alpha * s1)
      mun[i] = sigma[i] @ ( s2 + lamb @ mu)
    
    #Sampling U
  
    for i in range(size):
      uv[i] = np.random.multivariate_normal(mun[i], sigma[i])
    spl = uv
    print(spl[t])
    # for i in range(size):
    #   uut[i] = U[i, :].reshape(D, 1) @ U[i ,:].reshape(1,D)
    return spl
def sample_v(lamb,mu,uv,uv2,size,size2):
    sigma = np.zeros((size, D, D))
    mun = np.zeros((size, D))
    for j in range(size): 
      s1 = np.zeros((D,D))
      s2 = np.zeros(D)
      for i in range(size2):
        s1 += I[i][j] * uv2[i, :].reshape(D, 1) @ uv2[i ,:].reshape(1,D)#vut[j]
        s2 += R[i][j] * uv2[i ,:]#uv2[i, :].reshape(D, 1) @ U[i ,:].reshape(1,D)
#uv2[j ,:]
      
      s2 *= alpha
      sigma[j] = np.linalg.pinv(lamb + alpha * s1)
      mun[j] = sigma[j] @ ( s2 + lamb @ mu)
    
    #Sampling U
  
    for i in range(size):
      uv[i] = np.random.multivariate_normal(mun[i], sigma[i])
    spl = uv
    print(spl[t])
    # for i in range(size):
    #   uut[i] = U[i, :].reshape(D, 1) @ U[i ,:].reshape(1,D)
    return spl


In [0]:
T = 100
U_spl = np.zeros((T ,N , D))
V_spl = np.zeros((T ,M ,D))
for t in range(T):

  #sample log and mu
  lambu,muu = sample_mu(N,U,lambu,muu)
  #print(lambu)
  #sample log and mu
  lambv,muv = sample_mu(M,V,lambv,muv)

  #sample U
  U_spl[t] = sample_u(lambu,muu,U,V,N,M)
  V_spl[t] = sample_v(lambv,muv,V,U,M,N)
  print (t + 1)
  predictR = np.zeros((N,M))
  predictR += U_spl[t]@V_spl[t].T
  print(predictR)
  RMSE = (np.sum(((ITest*predictR) - RTest) ** 2) / np.sum(ITest)) ** 0.5
  print(RMSE)

[-1.4713551  -1.4561111   0.53010859  0.08178807  0.02132132  0.806325
 -0.56314941  1.67429604 -0.20121779 -0.77969724 -0.43481091 -0.31661084
  1.68595379 -0.89252587 -0.98471954  0.17153333  0.9594806   0.57905774
  1.43526355 -0.24766978 -0.17960006 -1.41602098  0.90822037  2.85659318
  1.64856462 -1.40181524  1.9469802  -1.80814997 -1.79881124  3.56562302
  0.10019048 -1.24979963  0.21468621  0.71219619 -0.72706254  0.88993962
  0.39381166  1.07104053 -0.80185257  0.92278961  0.10321413  3.0389393
 -0.11418408  0.79478127  1.8502203  -0.1742113  -2.22596288  1.07462453
  0.05319862  0.75414058] mu
[-5.59804548e-01  2.34416497e+00 -5.61930136e+00 -5.19482604e+00
  9.39808913e+00  5.37966750e+00 -8.40259866e-01  1.03218114e+00
  6.45461059e+00  5.10153946e+00  2.07788653e+00  1.42499687e+00
  3.76645558e+00  6.35869728e-01 -3.49744039e+00 -2.03550166e+00
  3.07098222e-02  2.58164927e+00 -3.01824133e+00  1.98360676e+00
  3.65087784e+00 -2.48085448e-01  2.00767013e+00  7.49137761e+00
